In [1]:
#Import Necessary Modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [2]:
source = requests.get('https://www.btk.gov.tr/yillik-il-istatistikleri')

In [3]:
soup = BeautifulSoup(source.text, 'lxml')

In [4]:
files = soup.find_all(name='div', attrs={'class':'SmTqk'})[0]

In [5]:
links = {}
for a in files.find_all(name='a'):
    link = a['href']
    if link.endswith('.xlsx'):
        links[a.text] = link

In [6]:
links

{'2016 - 2021': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-v1.xlsx',
 '2015 - 2020': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/elektronik-haberlesme-sektorune-iliskin-il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2014 - 2019': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-20-05-2020.xlsx',
 '2013 - 2018': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni1.xlsx',
 '2012 - 2017': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2011 - 2016': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2011-2016-5a3a48147c468.xlsx',
 '2010 - 2015': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2010-2015.xlsx',
 '2009 - 2014': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2009-2014.xlsx',
 '2008 - 2013': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2

In [7]:
def get_dataframe(link):
    data = pd.read_excel(link, sheet_name='İl Bazlı Veri')
    data = data.iloc[2:,:]
    data['detail'] = data.iloc[:,1].fillna('') + data.iloc[:,2].fillna('')
    data['city'] = data.iloc[:,0]
    data = data.iloc[:,3:]
    data['city'] = data['city'].fillna(method='ffill')
    data.dropna(subset=[x for x in data.columns if str(x) not in ['city','detail']], axis=0, inplace=True)
    data['detail'] = data['detail'].str.strip(r'[()]')
    return data[data.columns[::-1]].reset_index(drop=True)[:-1]

In [37]:
df = pd.DataFrame()
for key, link in links.items():
    print(key)
    data = get_dataframe(link)
    if df.shape[0] == 0:
        df = data
    else:
        cols_to_use = data.columns.difference(df.columns[2:])
        df = df.merge(data[cols_to_use], how='outer', on=['city', 'detail'])

2016 - 2021
2015 - 2020
2014 - 2019
2013 - 2018
2012 - 2017
2011 - 2016
2010 - 2015
2009 - 2014
2008 - 2013
2007 - 2012


In [41]:
df.sort_values(by='city', inplace=True)

In [42]:
df.to_csv('final_merged.csv', encoding='utf-8-sig')

In [43]:
df.columns

Index([  'city', 'detail',     2021,     2020,     2019,     2018,     2017,
           2016,     2015,     2014,     2013,     2012,     2011,     2010,
           2009,     2008,     2007],
      dtype='object')